# Experiment 1. Extracting finger-onset asynchrony: response time brackets for two- and three-target disappearance and appearance events

In [10]:
import os
import pandas as pd
import numpy as np

### change in physical state
State= pd.read_csv(os.path.join('Experiment 1', 'Experiment1_FOA_DisplayState.csv'), sep=';')
State['TargetCount']=State.Target1+State.Target2+State.Target3
State['Trial']= int(0)
State['ChangeMagnitude']= int(0)
for current_ID in State.ID.unique():
    for current_block in State[State.ID==current_ID].Block.unique():
        block_trials= State[(State.ID==current_ID) & (State.Block==current_block)].index
        for iTrial, iRow in enumerate(block_trials):
            State.set_value(iRow, 'Trial', iTrial)
            if iTrial>0:
                State.set_value(iRow, 'ChangeMagnitude', State.TargetCount[iRow]-State.TargetCount[iRow-1])     
                
# splitting signed change magnitude into event type (appearance vs. disappearance) and absolute change magnitude
State['EventType']= 'appearance'
State.set_value(State[State.ChangeMagnitude<0].index, 'EventType', 'disappearance')
State['ChangeMagnitude']= State['ChangeMagnitude'].abs()

### Loadng responses
Response= pd.read_csv(os.path.join('Experiment 1', 'Experiment1_FOA_Response.csv'), sep=';')

### computing response time delay and response time difference for each State Change
State['FirstResponse']= np.nan
State['LastResponse']= np.nan
State['ResponseCount']= int(0)
for iRow in range(len(State)):
    # skipping the first trial, since this the display onset rather than a change
    if np.isnan(State.ChangeMagnitude[iRow]):
        continue
        
    # identifying response time window between state changes
    response_start= State.ChangeTime[iRow]
    if iRow==len(State)-1: # last trial of the last block
        response_end= State.Duration[iRow]
    elif State.Block[iRow+1]!=State.Block[iRow]: # last trial of the block
        response_end= State.Duration[iRow]
    else: # normal trial
        response_end= State.ChangeTime[iRow+1]
        
    # identifying reponses
    iResponse= Response[(Response.ID==State.ID[iRow]) & (Response.Block==State.Block[iRow]) & (Response.Time>=response_start) & (Response.Time<response_end)].index
    State.set_value(iRow, 'ResponseCount', len(iResponse))
    if len(iResponse)>0:
        State.set_value(iRow, 'FirstResponse', Response.Time[iResponse[0]]-response_start)
        State.set_value(iRow, 'LastResponse', Response.Time[iResponse[-1]]-response_start)
    else:
        State.set_value(iRow, 'FirstResponse', np.nan)
        State.set_value(iRow, 'LastResponse', np.nan)

# computing time bracket for multikey reponses
State['TimeBracket']= State.LastResponse-State.FirstResponse        

# identifying valid trials (numbers of presses/releases matches the number of state changes)
State['Valid']= State.ChangeMagnitude==State.ResponseCount

### computing average response time brackets for two and three event
ResponseBrackets= State[(State.Valid) & (State.ChangeMagnitude>1)].groupby(['ID', 'EventType', 'ChangeMagnitude']).apply(
    lambda x: pd.Series({'MeanRT': x.TimeBracket.mean(),
                         'MinRT': x.TimeBracket.min(),
                         'MaxRT': x.TimeBracket.max(),
                         'StdRT': x.TimeBracket.std()})).reset_index()
ResponseBrackets.to_csv(os.path.join('Experiment 1', 'Experiment1_FOA_Estimate.csv'), sep=';', index=False)